
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GP211/2023-fall-class-notebooks/blob/main/in-class/008-Pre-1d.ipynb)

In [ ]:
! python3 -m pip install  ! pip install  "giee @ git+https://github.com/SEP-software/2023-fall-class-notebooks.git.git@d00084220a6501d5fe744869f82e64b3dab9c03b" 
%load_ext autoreload
%autoreload 2
import sys

In [ ]:
import numba
from generic_solver import Operator
from generic_solver._pyOperator import ChainOperator
from sep_python import FloatVector,get_sep_vector
import numpy as np
from giee import Tconv,Jop


In [ ]:
from generic_solver import ProblemL2Linear,ProblemL2LinearReg
from generic_solver import LCGsolver, BasicStopper
from giee import Causal, CausalBoth
import holoviews as hv
from sep_plot import Dots



In [ ]:
from generic_solver._pyOperator import DiagonalOp
data_fit=start_model.clone()
tconv_op=Tconv(model,data_reg,filt)

jop=DiagonalOp(get_sep_vector(kn))
model.zero()
prob_reg=ProblemL2LinearReg(model,data_fit,jop,reg_op=tconv_op,epsilon=1.)
stop=BasicStopper(niter=nm[0])

solve_reg=LCGsolver(stop)
solve_reg.setDefaults(save_model=True)
solve_reg.run(prob_reg)
Graph(data_fit,width=700)+Graph(prob_reg.model,width=700)

In [ ]:
import holoviews as hv
from sep_plot import Graph
from giee import Tconv
from generic_solver._pyOperator import DiagonalOp
hv.extension('bokeh','matplotlib')
fill = np.cos(np.linspace(start = -np.pi*(3/2), stop = 5*np.pi*(3/2), num = 300)).astype(np.float32)
sc=np.zeros((300))
sc[:]=1
x=(300*(np.sqrt(np.random.rand(200)))).astype(np.int32)
for i in x:
    fill[i]=0
inp=get_sep_vector(fill)
filt=np.array([1,-1.])
model=inp.clone()
start_model=model.clone()
nm=model.get_hyper().get_ns()
data_reg=get_sep_vector(ns=[nm[0]+filt.shape[0]-1])
unknown=model.clone()
known=model.clone()
un=unknown.get_nd_array()
kn=known.get_nd_array()
un[:]=1
kn[:]=0
for ival,val in enumerate(model.get_nd_array()):
    if val !=0:
      kn[ival]=1
      un[ival]=0

op=Tconv(model,data_reg,filt)
prob_base=ProblemL2Linear(model,data_reg,op,grad_mask=unknown)
stop=BasicStopper(niter=nm[0])

solve_base=LCGsolver(stop)
solve_base.setDefaults(save_model=True)
solve_base.run(prob_base)

Graph(model)+Graph(prob_base.model)

In [ ]:
data_fit=start_model.clone()
jop=DiagonalOp(get_sep_vector(kn))
ones=np.ones((300))
iop=DiagonalOp(get_sep_vector(ones))

causal=Causal(model,model)
op_fit=ChainOperator(causal,jop)
model.zero()
op_fit.dotTest()
prob_prec=ProblemL2LinearReg(model,data_fit,op_fit,reg_op=iop,epsilon=.000000)
stop=BasicStopper(niter=100)

solve_prec=LCGsolver(stop)
solve_prec.setDefaults(save_model=True)
solve_prec.run(prob_prec)
tmp=prob_prec.model.clone()

prob_prec.op.forward(False,prob_prec.model,prob_prec.res)
causal.forward(False,prob_prec.model,tmp)
Graph(prob_prec.data)+Graph(tmp)



In [ ]:
print(mod.max(),mod.min())

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
def plot_result_reg(iter):
    return (Graph(data_fit,width=400)+Graph(solve_reg.modelSet.vecSet[iter],width=400)).cols(1)

# When run live, this cell's output should match the behavior of the GIF below
dmap = hv.DynamicMap(plot_result_reg, kdims=['iter'])
dmap.redim.range(iter=(0,len(solve_reg.modelSet.vecSet)-1))
#dmap = dmap.redim.values(Method=methods)


In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
def plot_result_prec(iter):
    tmp=prob_prec.model.clone()
    causal.forward(False,solve_prec.modelSet.vecSet[iter],tmp)
    return (Graph(data_fit,width=700)+Graph(tmp,width=700)).cols(1)

# When run live, this cell's output should match the behavior of the GIF below
dmap = hv.DynamicMap(plot_result_prec, kdims=['iter'])
dmap.redim.range(iter=(0,50))